<a href="https://colab.research.google.com/github/mesuttguven/ML4CS/blob/main/ML4CS_w11_Gpt2_Malware_detection_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SPECIAL NOTE:**

This Colab belongs to *Nazenin Sahin (M.S. in Cyber Security)* who is a distinguished colleague of mine. 

The model was created during her dissertation thesis and M.S. studies.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git

In [ ]:
!pip install torch==1.7.0 #set_seed error and model = RobertaForSequenceClassification row error
!pip install transformers==3.5.1 

In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

# Set seed for reproducibility.
set_seed(82)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 2

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 64

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 45

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
#model_name_or_path = './drive/My Drive/TEZ/GPT2/GPT2_3000000/GPT2_3000000_byte_model'


In [ ]:
!wc -l './drive/My Drive/own_data/malicious.txt'



In [ ]:
!head -n 5000000 './drive/My Drive/own_data/benign.txt' > './drive/My Drive/data_5/benign.txt'

In [ ]:
class MalBengDataset(Dataset):
  r"""PyTorch Dataset class for loading data.

  This is where the data parsing happens.

  This class is built with reusability in mind: it can be used as is as.

  Arguments:

    path (:obj:`str`):
        Path to the data partition.

  """

  def __init__(self, path, use_tokenizer):

    self.texts = []
    self.labels = []

    

    DATA_DIR=path
    MERGED_MAL_FILE=os.path.join(DATA_DIR,'malicious.txt')
    print(MERGED_MAL_FILE)
    MERGED_BENIGN_FILE=os.path.join(DATA_DIR,'benign.txt')
    print(MERGED_BENIGN_FILE)

    FILE_NAMES = [file for file in os.listdir(DATA_DIR)]
    print(FILE_NAMES)
    for i, file_name in enumerate(FILE_NAMES):
        with open(os.path.join(DATA_DIR,file_name), "r") as f:
            for line in f:
                self.texts.append(line)
                self.labels.append(i)
        print(i)
        print(file_name)

    # Number of exmaples.
    self.n_examples = len(self.labels)
    

    return

  def __len__(self):
    r"""When used `len` return the number of examples.

    """
    
    return self.n_examples

  def __getitem__(self, item):
    r"""Given an index return an example from the position.
    
    Arguments:

      item (:obj:`int`):
          Index position to pick an example to return.

    Returns:
      :obj:`Dict[str, str]`: Dictionary of inputs that contain text and 
      asociated labels.

    """

    return {'text':self.texts[item],
            'label':self.labels[item]}





In [ ]:
class GPT2ClassificationCollator(object):
    r"""
    Data Collator used for GPT2 in a classificaiton rask. 
    
    It uses a given tokenizer and label encoder to convert any text and labels to numbers that 
    can go straight into a GPT2 model.

    This class is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

    Arguments:

      use_tokenizer (:obj:`transformers.tokenization_?`):
          Transformer type tokenizer used to process raw text into numbers.

      labels_ids (:obj:`dict`):
          Dictionary to encode any labels names into numbers. Keys map to 
          labels names and Values map to number associated to those labels.

      max_sequence_len (:obj:`int`, `optional`)
          Value to indicate the maximum desired sequence to truncate or pad text
          sequences. If no value is passed it will used maximum sequence size
          supported by the tokenizer and model.

    """

    def __init__(self, use_tokenizer, max_sequence_len=None):

        # Tokenizer to be used inside the class.
        self.use_tokenizer = use_tokenizer
        # Check max sequence length.
        self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len

        return

    def __call__(self, sequences):
        r"""
        This function allowes the class objesct to be used as a function call.
        Sine the PyTorch DataLoader needs a collator function, I can use this 
        class as a function.

        Arguments:

          item (:obj:`list`):
              List of texts and labels.

        Returns:
          :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
          It holddes the statement `model(**Returned Dictionary)`.
        """

        # Get all texts from sequences list.
        texts = [sequence['text'] for sequence in sequences]
        # Get all labels from sequences list.
        labels = [sequence['label'] for sequence in sequences]
        # Call tokenizer on all texts to convert into tensors of numbers with 
        # appropriate padding.
        inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
        # Update the inputs with the associated encoded labels as tensor.
        inputs.update({'labels':torch.tensor(labels)})

        #print(inputs)
        return inputs


def train(dataloader, optimizer_, scheduler_, device_):

  # Use global variable for model.
  global model

  # Tracking variables.
  predictions_labels = []
  true_labels = []
  # Total loss for this epoch.
  total_loss = 0

  # Put the model into training mode.
  model.train()

  # For each batch of training data...
  for batch in tqdm(dataloader, total=len(dataloader)):

    # Add original labels - use later for evaluation.
    true_labels += batch['labels'].numpy().flatten().tolist()
    
    # move batch to device
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
    
    # Always clear any previously calculated gradients before performing a
    # backward pass.
    model.zero_grad()

    # Perform a forward pass (evaluate the model on this training batch).
    # This will return the loss (rather than the model output) because we
    # have provided the `labels`.
    # The documentation for this a bert model function is here: 
    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    outputs = model(**batch)

    # The call to `model` always returns a tuple, so we need to pull the 
    # loss value out of the tuple along with the logits. We will use logits
    # later to calculate training accuracy.
    loss, logits = outputs[:2]

    # Accumulate the training loss over all of the batches so that we can
    # calculate the average loss at the end. `loss` is a Tensor containing a
    # single value; the `.item()` function just returns the Python value 
    # from the tensor.
    total_loss += loss.item()

    # Perform a backward pass to calculate the gradients.
    loss.backward()

    # Clip the norm of the gradients to 1.0.
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)

    # Update parameters and take a step using the computed gradient.
    # The optimizer dictates the "update rule"--how the parameters are
    # modified based on their gradients, the learning rate, etc.
    optimizer.step()

    # Update the learning rate.
    scheduler.step()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Convert these logits to list of predicted labels values.
    predictions_labels += logits.argmax(axis=-1).flatten().tolist()

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)
  
  # Return all true labels and prediction for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss



def validation(dataloader, device_):
  r"""Validation function to evaluate model performance on a 
  separate set of data.

  This function will return the true and predicted labels so we can use later
  to evaluate the model's performance.

  This function is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

  Arguments:

    dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
          Parsed data into batches of tensors.

    device_ (:obj:`torch.device`):
          Device used to load tensors before feeding to model.

  Returns:
    
    :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
        Labels, Train Average Loss]
  """

  # Use global variable for model.
  global model

  # Tracking variables
  predictions_labels = []
  true_labels = []
  #total loss for this epoch.
  total_loss = 0

  # Put the model in evaluation mode--the dropout layers behave differently
  # during evaluation.
  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dataloader, total=len(dataloader)):

    # add original labels
    true_labels += batch['labels'].numpy().flatten().tolist()

    # move batch to device
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to to calculate training accuracy.
        loss, logits = outputs[:2]
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        
        # get predicitons to list
        predict_content = logits.argmax(axis=-1).flatten().tolist()

        # update list
        predictions_labels += predict_content

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediciton for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss

In [ ]:
# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='./drive/My Drive/GPT2/Gpt2_son_gpt/Gpt2_son_model/gpt2_28_06/checkpoint-600000', num_labels=2)
#model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='EleutherAI/gpt-neo-1.3B', num_labels=2)


#model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='gpt2-medium', num_labels=2)
# Get model's tokenizer.
print('Loading tokenizer...')
#tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='EleutherAI/gpt-neo-1.3B')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='./drive/My Drive/GPT2/Gpt2_son_gpt/Gpt2_son_config')
special_tokens_dict = {"bos_token": '<bos>', "eos_token": '<eos>', "unk_token": '<unk>', "sep_token": '<sep>',
                                 "pad_token":'<pad>', "cls_token": '<cls>', "mask_token" : '<mask>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)


# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='./drive/My Drive/GPT2/Gpt2_son_gpt/Gpt2_son_model/gpt2_28_06/checkpoint-600000', config=model_config)
#model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='EleutherAI/gpt-neo-1.3B', config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

In [ ]:
model.cuda()

In [ ]:
#tokenizer.special_tokens_map

In [ ]:
#special_tokens_dict = {'pad_token': '<PAD>'}
#num_added_toks = tokenizer.add_special_tokens(SPECIAL_TOKENS_ATTRIBUTES)
#model.resize_token_embeddings(len(tokenizer))

In [ ]:
#model.config.pad_token_id

In [ ]:
tokenizer.all_special_tokens

In [ ]:
tokenizer.all_special_ids

In [ ]:
tokenizer.encode("sub eax,DWORD PTR [eax]")

In [ ]:
sent="sub eax,DWORD PTR [eax]"

In [ ]:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 45,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

In [ ]:
encoded_dict.

In [ ]:
encoded_dict

In [ ]:
#tokenizer.sep_token

In [ ]:
#model.cuda()>

In [ ]:

#from sklearn import datasets, linear_model
#from sklearn.model_selection import train_test_split
#from matplotlib import pyplot as plt

In [ ]:
GPT2_classificaiton_collator = GPT2ClassificationCollator(use_tokenizer=tokenizer, max_sequence_len=max_length)


print('Dealing with Train...')
# Create pytorch dataset.
gpt2_dataset = MalBengDataset(path='./drive/My Drive/data_6', use_tokenizer=tokenizer)
print('Created `all_dataset` with %d examples!'%len(gpt2_dataset))


#X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

total_count=len(gpt2_dataset) #splt data for train,validation and train
train_count = int(0.8 * total_count)
print('Created `train_dataset` with %d examples!'%(train_count))
valid_count = int(0.2 * total_count)
print('Created `valid_count` with %d examples!'%(valid_count))

train_dataset, valid_dataset= torch.utils.data.random_split(
    gpt2_dataset, (train_count, valid_count))#+1 ekledim neden?anlamadım


# Move pytorch dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=GPT2_classificaiton_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))


print('Dealing with Validation...')
# Create pytorch dataset.
#valid_dataset =  MalBengDataset(path='./drive/My Drive/TEZ/test', 
                               #use_tokenizer=tokenizer)
print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move pytorch dataset into dataloader.
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=GPT2_classificaiton_collator)
print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

In [ ]:
max_len_gpt2 = 0


# For every data_sets element...
for sent in data_set:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids_gpt2 = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum data_set length.
    max_len_gpt2 = max(max_len_gpt2, len(input_ids_gpt2))
    
print('Max sentence length GPT2: ', max_len_gpt2)
#Max sentence length gpt2:  34

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}

# Loop through each epoch.
print('Epoch')
for epoch in tqdm(range(epochs)):
  print()
  print('Training on batches...')
  # Perform one full pass over the training set.
  train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)

  # Get prediction form model on validation data. 
  print('Validation on batches...')
  valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
  val_acc = accuracy_score(valid_labels, valid_predict)

  # Print loss and accuracy values to see how training evolves.
  print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
  print()

  # Store the loss value for plotting the learning curve.
  all_loss['train_loss'].append(train_loss)
  all_loss['val_loss'].append(val_loss)
  all_acc['train_acc'].append(train_acc)
  all_acc['val_acc'].append(val_acc)

 

In [ ]:
12 milyon data,batch_size=64,3 epochs,clip_grad=1,0 

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
labels_ids = {'neg': 0, 'pos': 1}

In [ ]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3

In [ ]:
# Get prediction form model on validation data. This is where you should use
# your test data.
true_labels, predictions_labels, avg_epoch_loss = validation(valid_dataloader, device)

# Create the evaluation report.
evaluation_report = classification_report(true_labels, predictions_labels, labels=list(labels_ids.values()), target_names=list(labels_ids.keys()))
# Show the evaluation report.
print(evaluation_report)

# Plot confusion matrix.
plot_confusion_matrix(y_true=true_labels, y_pred=predictions_labels, 
                      classes=list(labels_ids.keys()), normalize=True, 
                      magnify=0.1,
                      );

In [ ]:
from sklearn.metrics import matthews_corrcoef

In [ ]:
sklearn.metrics.matthews_corrcoef(true_labels, predictions_labels, sample_weight=None )